In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.layers import LSTM, Dense, Dropout, Input, concatenate, Activation
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
from datetime import datetime

# establish sql connection
db_path = "drive/MyDrive/data/input_data.db"
conn = sqlite3.connect(db_path)

In [ ]:
parser = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

df_station43_long = pd.read_sql_query('SELECT Date, solar_radiation, pressureTrend, windspeedAvg, avg_wind_dir '
                                      'FROM wunderground_historical_43_long', conn,
                                      parse_dates=['Date'], index_col='Date')
df_station43_long.dropna(inplace=True)
# df_station43_long = df_station43_long.resample('5Min').mean().interpolate(method='linear')

df_mb_15 = pd.read_sql_query('SELECT Timestamp, pvpower_instant FROM mb_pvpro_15min', conn, 
                             parse_dates=['Timestamp'], index_col='Timestamp')
df_mb_15.dropna(inplace=True)
df_mb_15 = df_mb_15.resample('5Min').mean().interpolate(method='linear')

df_mb_60 = pd.read_sql_query('SELECT Timestamp, pvpower_instant FROM mb_pvpro_1h', conn, 
                             parse_dates=['Timestamp'], index_col='Timestamp')
df_mb_60.dropna(inplace=True)
df_mb_60 = df_mb_60.resample('5Min').mean().interpolate(method='linear')
df_mb_pv = pd.concat([df_mb_60, df_mb_15])

df_mb_clouds = pd.read_sql_query('SELECT Timestamp, lowclouds, midclouds, highclouds, totalcloudcover '
                                 'FROM mb_clouds', conn, parse_dates=['Timestamp'], index_col='Timestamp')
df_mb_clouds.dropna(inplace=True)
df_mb_clouds = df_mb_clouds.resample('5Min').mean().interpolate(method='linear')

# convert dtypes to reduce RAM usage
df_station43_long['solar_radiation'] = df_station43_long['solar_radiation'].astype(np.float32)
df_station43_long['avg_wind_dir'] = df_station43_long['avg_wind_dir'].astype(np.int16)
df_station43_long['windspeedAvg'] = df_station43_long['windspeedAvg'].astype(np.int8)
df_station43_long['pressureTrend'] = df_station43_long['pressureTrend'].astype(np.float16)


In [ ]:
df_station43_long = df_station43_long.resample('5Min').mean().interpolate(method='quadratic')

df_station43_long['day'] = df_station43_long.index.day.astype(np.int8)
df_station43_long['month'] = df_station43_long.index.month.astype(np.int8)
df_station43_long['hour'] = df_station43_long.index.hour.astype(np.int8)
df_station43_long['minute'] = df_station43_long.index.minute.astype(np.int8)

handle missing data (09.05.2021 - 26.05.2021)

In [ ]:
# interpolate small data gaps
split_date = pd.to_datetime('25.09.2022', format='%d.%m.%Y')

data_1 = df_station43_long.loc[split_date:]
data_2 = df_station43_long.loc[pd.to_datetime('26.05.2021', format='%d.%m.%Y'):split_date]

# change structure so date values are always in the first four spots
cols = data_1.columns.tolist()
cols = cols[-4:] + cols[:-4]
data_1 = data_1[cols]
data_2 = data_2[cols]

# merge forecast data
data_1 = data_1.merge(df_mb_pv, how='inner', left_index=True, right_index=True)
data_1 = data_1.merge(df_mb_clouds, how='inner', left_index=True, right_index=True)

data_2 = data_2.merge(df_mb_pv, how='inner', left_index=True, right_index=True)
data_2 = data_2.merge(df_mb_clouds, how='inner', left_index=True, right_index=True)

# remove negative values for scaling
data_2.loc[data_2['solar_radiation'] < 0, 'solar_radiation'] = 0
data_2.loc[data_2['pvpower_instant'] < 0, 'pvpower_instant'] = 0

data_1.loc[data_1['solar_radiation'] < 0, 'solar_radiation'] = 0
data_1.loc[data_1['pvpower_instant'] < 0, 'pvpower_instant'] = 0

# maximum solar radiation should be 1000
# print(data_1.max())
data_1.loc[data_1['solar_radiation'] > 0, 'solar_radiation'] = 1000
data_2.loc[data_2['solar_radiation'] > 0, 'solar_radiation'] = 1000

In [ ]:
# scale the data using MinMax Scaler from to 1 as LSTM has a default tanh activation function

test_data = pd.concat([data_1, data_2])
test_scaler = [MinMaxScaler(feature_range=(-1,1)).fit(test_data.to_numpy().T[i].reshape(-1, 1)) for i in range(test_data.shape[-1])]

# scalers_1 = [MinMaxScaler(feature_range=(-1,1)).fit(data_1.to_numpy().T[i].reshape(-1, 1)) for i in range(data_1.shape[-1])]
# scalers_2 = [MinMaxScaler(feature_range=(-1,1)).fit(data_2.to_numpy().T[i].reshape(-1, 1)) for i in range(data_2.shape[-1])]

data_1_scaled = np.array([scaler.transform(data_1.to_numpy().T[i].reshape(-1, 1)) for i, scaler in enumerate(test_scaler)]).squeeze()
data_2_scaled = np.array([scaler.transform(data_2.to_numpy().T[i].reshape(-1, 1)) for i, scaler in enumerate(test_scaler)]).squeeze()

In [ ]:
# create a function to split the datasets into two week windows
timesteps_input = 12*24  # 12 five min intervals * 4 hours 
timesteps_prediction = 12*4  # 12 five min intervals * 2 hours

def create_dataset(dataset, steps_in=timesteps_input, steps_pred=timesteps_prediction):
    """
    Function which creates two week chunks of x_train data, and a single
    value for y_train.
    """
    X_hist, X_pred, y = [], [], []
    print(dataset.shape)
    for i in tqdm(range(dataset.shape[1])):
        target_val_start = i + steps_in
        target_val_end = target_val_start + steps_pred
        if target_val_end >= dataset.shape[1]:
            break
        feature_chunk, meteoblue_pred, target = dataset[:-5, i:target_val_start], \
                                                dataset[-5:, target_val_start:target_val_end], \
                                                dataset[4, target_val_start:target_val_end]
        X_hist.append(feature_chunk)
        X_pred.append(meteoblue_pred)
        y.append(target)

    return np.array(X_hist), np.array(X_pred), np.array(y)

In [ ]:
# create training data for NN
X_hist_1, X_pred_1, y_1 = create_dataset(data_1_scaled)
X_hist_2, X_pred_2, y_2 = create_dataset(data_2_scaled)

In [ ]:
print(X_hist_1.shape)
print(X_pred_1.shape)
print(y_1.shape)
print(X_hist_2.shape)
print(X_pred_2.shape)
print(y_2.shape)

Create LSTM Model

In [ ]:
# input needs to be [samples, timesteps, features]
units_1 = 64
units_2 = 96
units_3 = 64
units_4 = 64
units_dense = 64
dropout = 0.0
epochs = 40
# val_split = 0.15
optimizer = 'adam'
loss='mse'
file_name = 'wg_multi_4hour_branches_mb_forecast_L64rs_L96rs_L64_D64_C_D64_hard_validation_mse_changed_split'

In [ ]:
# multiple inputs from https://pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/
input_hist = Input(shape=(X_hist_2.shape[1], X_hist_2.shape[2]))
input_pred = Input(shape=(X_pred_2.shape[1], X_pred_2.shape[2]))

# branch for historical wonderground data
x = LSTM(units_1, dropout=dropout, return_sequences=True)(input_hist)
x = LSTM(units_2, return_sequences=True)(x)
x = LSTM(units_3)(x)
x = Dense(units_4, activation='relu')(x)
x = Model(inputs=input_hist, outputs=x)

# branch for meteoblue forecast data
y = LSTM(units_1, dropout=dropout, return_sequences=True)(input_pred)
y = LSTM(units_2, return_sequences=True)(y)
y = LSTM(units_3)(y)
y = Dense(units_4, activation='relu')(y)
y = Model(inputs=input_pred, outputs=y)

# combine branches
combined = concatenate([x.output, y.output])
z = Dense(units_dense, activation='relu')(combined)
# z = LSTM(64)(z)
z = Dense(y_2.shape[1])(z)

model = Model(inputs=[x.input, y.input], outputs=z)

checkpoint_filepath = 'drive/MyDrive/data/{}_cp'.format(file_name)
mcp_save = keras.callbacks.ModelCheckpoint(checkpoint_filepath, save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(patience=8, monitor='val_loss', restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss)
print(model.summary())

history = model.fit([X_hist_2, X_pred_2], y_2, validation_data=([X_hist_1, X_pred_1], y_1), batch_size=300,
                    epochs=epochs, verbose=1, callbacks=[early_stopping])

# batchsize größer

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epoch = np.arange(1, len(val_loss)+1, 1)

fig = plt.figure(figsize=(12, 8))
plt.plot(epoch,loss)
plt.plot(epoch,val_loss)
plt.show()

In [ ]:
model.save('drive/MyDrive/data/{}'.format(file_name))

In [ ]:
model = keras.models.load_model('drive/MyDrive/data/{}'.format(file_name))

In [ ]:
print(model.summary())

In [ ]:
# plot relevant times (7am - 2pm)

while True:
    r = random.randrange(0, len(X_hist_1))
    x_hist = X_hist_1[r]
    x_hist_exp = np.expand_dims(x_hist, 0)
    x_pred = X_pred_1[r]
    x_pred_exp = np.expand_dims(x_pred, 0)
    expected = y_1[r]
    prediction = model([x_hist_exp, x_pred_exp], training=False)[0]

    x_hist_real = np.array([scaler.inverse_transform(x_hist[i].reshape(-1, 1)) for i, scaler in enumerate(test_scaler[:-5])])
    x_pred_real = np.array([scaler.inverse_transform(x_pred[i].reshape(-1, 1)) for i, scaler in enumerate(test_scaler[-5:])])
    expected_real = test_scaler[4].inverse_transform(expected.reshape(-1, 1))
    prediction_real = test_scaler[4].inverse_transform(prediction.numpy().reshape(-1, 1))

    if int(x_hist_real[2][-1]) in range(7, 12, 1):
        break


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 9))
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

axes[0].set_title('Scaled Values - Date: {:02d}.{:02d}. {:02d}:{:02d}'.format(int(x_hist_real[0][-1]), int(x_hist_real[1][-1]), 
                                                       int(x_hist_real[2][-1]), int(x_hist_real[3][-1])))
axes[0].plot(expected, label='Expected')
axes[0].plot(prediction, label='Prediction')
axes[0].plot(x_pred[0], label='MeteoBlue PV-Forecast')
# axes[0].plot(x_pred[-1], label='MeteoBlue Cloud-Forecast')
axes[0].legend()
axes[0].grid()

print('MSE MeteoBlue: {:.4f}'.format(mean_squared_error(expected, x_pred[0])))
print('MSE Prediction: {:.4f}'.format(mean_squared_error(expected, prediction)))
print('\n')

axes[1].set_title('Real Values - Date: {:02d}.{:02d}. {:02d}:{:02d}'.format(int(x_hist_real[0][-1]), int(x_hist_real[1][-1]), 
                                                       int(x_hist_real[2][-1]), int(x_hist_real[3][-1])))
axes[1].plot(expected_real, label='Expected')
axes[1].plot(prediction_real, label='Prediction')
axes[1].plot(x_pred_real[0], label='MeteoBlue PV-Forecast')
# axes[1].plot(x_pred_real[-1], label='MeteoBlue Cloud-Forecast')
axes[1].legend()
axes[1].grid()

print('Real MSE MeteoBlue: {:.4f}'.format(mean_squared_error(expected_real, x_pred_real[0])))
print('Real MSE Prediction: {:.4f}'.format(mean_squared_error(expected_real, prediction_real)))
print('\n')

print('Meteoblue max/min: {}/{}'.format(max(x_pred_real[0]), min(x_pred_real[0])))
print('Expected max/min: {}/{}'.format(max(expected_real), min(expected_real)))